In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
!git clone https://github.com/AllenWrong/10714-final-proj.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
%cd 10714-final-proj/
!pwd

/content/drive/MyDrive/10714-final-proj
/content/drive/MyDrive/10714-final-proj


In [ ]:
!pip install pybind11
!make

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 222 kB 4.8 MB/s 


## Import

In [ ]:
import sys
sys.path.append("python/")

In [ ]:
import needle as ndl
from needle import ops
import numpy as np

## Implement the op  `__getitem__` for Tensor

In [ ]:
a = np.array([
    [1, 2, 3, 4], 
    [2, 3, 4, 5], 
    [6, 7, 8, 9]
]).astype("float32")

### Torch example

In [ ]:
import torch

In [ ]:
th_tensor = torch.tensor(a, requires_grad=True)
b = th_tensor[1:3, 1:3]

b.sum().backward()
print(th_tensor.grad)

tensor([[0., 0., 0., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.]])


### Needle example

In [ ]:
import needle as ndl

In [ ]:
nd_tensor = ndl.Tensor(a)
b_nd = nd_tensor[1:3, 1:3]

b_nd.sum().backward()
print(nd_tensor.grad)

[[0. 0. 0. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]]


## Implement `ndl.cat([...], axis=)`

In [ ]:
a = np.array([
    [1, 2, 3, 4],
    [2, 3, 4, 5], 
    [6, 7, 8, 9]
]).astype("float32")

### Torch example

In [ ]:
th1 = torch.tensor(a[:, 0:2], requires_grad=True)
th2 = torch.tensor(a[:, 2:], requires_grad=True)

print(th1)
print(th2)

tensor([[1., 2.],
        [2., 3.],
        [6., 7.]], requires_grad=True)
tensor([[3., 4.],
        [4., 5.],
        [8., 9.]], requires_grad=True)


In [ ]:
# forward
th12 = torch.cat([th1, th2], axis=1)
th12

tensor([[1., 2., 3., 4.],
        [2., 3., 4., 5.],
        [6., 7., 8., 9.]], grad_fn=<CatBackward0>)

In [ ]:
# backward
th12.sum().backward()
print(th1.grad)
print(th2.grad)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


### Needle example

In [ ]:
nd1 = ndl.Tensor(a[:, 0:2])
nd2 = ndl.Tensor(a[:, 2:])

print(nd1)
print(nd2)

[[1. 2.]
 [2. 3.]
 [6. 7.]]
[[3. 4.]
 [4. 5.]
 [8. 9.]]


In [ ]:
# forward
nd12 = ops.cat([nd1, nd2], axis=1)
nd12

needle.Tensor([[1. 2. 3. 4.]
 [2. 3. 4. 5.]
 [6. 7. 8. 9.]])

In [ ]:
# backward
nd12.sum().backward()
print(nd1.grad)
print(nd2.grad)

[[1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]]


## Implement `inv`

Following is the formula about how we get the gradient of inv op

In matrix calculus, we have the following formulas.

$$
dXY = d(X)Y + XdY \\
XX^{-1} = I \\
dI = \mathbf{0}
$$

Using the above three formulas, we can get the calculus between $X^{-1}$ and $X$:

$$
d(X)X^{-1}+XdX^{-1} = \mathbf{0} \\
dX = -XdX^{-1}X \\
dX^{-1} = -X^{-1}d(X)X^{-1}
$$

The relationship between the calculus and derivation is (f: $R^{m\times n} → R$ is scalar function):

$$
df = tr((\frac{\partial f}{\partial X})^TdX)
$$

For the same scalar function, we can get the follow formula:

$$
df = tr((\frac{\partial f}{\partial X})^TdX) \\
df = tr((\frac{\partial f}{\partial X^{-1}})^TdX^{-1})
$$

Replace the $dX^{-1}$ using $-X^{-1}d(X)X^{-1}$, we can get:

$$
tr((\frac{\partial f}{\partial X})^T dX) =
tr((\frac{\partial f}{\partial X^{-1}})^T (-X^{-1})d(X)X^{-1})
$$

There exists a formula for trace: $tr(AB)=tr(BA)$. Using this formula, we can rewrite the above formula:

$$
tr((\frac{\partial f}{\partial X})^T dX) =
tr(X^{-1}(\frac{\partial f}{\partial X^{-1}})^T (-X^{-1})d(X))
$$

Which means:

$$
\frac{\partial f}{\partial X} = (X^{-1}(\frac{\partial f}{\partial X^{-1}})^T (-X^{-1}))^T
$$

Following this formula, we will implement our gradient method for **inv**.

**Using row operation, we implement the inv. Following is a demo program using numpy.**

In [ ]:
import numpy as np

x = np.array([
    [3., 2., 3.],
    [2., 3., 4.],
    [6., 7., 8.]
])

class linalg:
    @staticmethod
    def inv(x):
        aug = np.concatenate([x, np.eye(x.shape[0])], axis=1)
        row, col = x.shape

        for i in range(row):
            aug[i, :] = aug[i, :] / aug[i, i]
            for j in range(i + 1, row):
                aug[j, :] = aug[j, :] - aug[j, i] * aug[i, :]

        for i in range(row - 1, 0, -1):
            for j in range(i - 1, -1, -1):
                aug[j, :] -= aug[j, i] * aug[i, :]

        return aug[:, col:]


import time
# test inv
for _ in range(100):
    arr = np.eye(100) + np.random.rand(100, 100)
    start_time = time.time()
    B = np.linalg.inv(arr)
    print("np.inv:", (time.time() - start_time) / 1000)
    start_time = time.time()
    arr_inv = linalg.inv(arr)
    print("nd.inv:", (time.time() - start_time) / 1000)
    assert np.allclose(B, arr_inv)

np.inv: 7.816791534423828e-06
nd.inv: 7.545280456542968e-05
np.inv: 6.12020492553711e-07
nd.inv: 7.393813133239747e-05
np.inv: 5.559921264648437e-07
nd.inv: 7.050967216491699e-05
np.inv: 5.221366882324219e-07
nd.inv: 6.85122013092041e-05
np.inv: 8.411407470703125e-07
nd.inv: 7.318472862243652e-05
np.inv: 6.911754608154297e-07
nd.inv: 7.442283630371093e-05
np.inv: 6.337165832519531e-07
nd.inv: 6.9929838180542e-05
np.inv: 5.166530609130859e-07
nd.inv: 6.85129165649414e-05
np.inv: 5.178451538085938e-07
nd.inv: 7.515358924865722e-05
np.inv: 5.242824554443359e-07
nd.inv: 7.227182388305664e-05
np.inv: 5.438327789306641e-07
nd.inv: 7.105636596679687e-05
np.inv: 5.218982696533203e-07
nd.inv: 7.250499725341797e-05
np.inv: 5.428791046142578e-07
nd.inv: 7.27541446685791e-05
np.inv: 5.18798828125e-07
nd.inv: 0.00012793922424316406
np.inv: 1.5947818756103516e-06
nd.inv: 0.0001005094051361084
np.inv: 5.273818969726563e-07
nd.inv: 7.960987091064453e-05
np.inv: 6.487369537353516e-07
nd.inv: 8.34143161

### Torch example

In [ ]:
import torch

In [ ]:
# example array
x = np.array([
    [3., 2., 3.],
    [2., 3., 4.],
    [6., 7., 8.]
])

In [ ]:
a_tensor = torch.tensor(x, requires_grad=True)
print(a_tensor.grad)

None


In [ ]:
b = torch.inverse(a_tensor)
b.retain_grad()
print(b)

tensor([[ 0.5000, -0.6250,  0.1250],
        [-1.0000, -0.7500,  0.7500],
        [ 0.5000,  1.1250, -0.6250]], dtype=torch.float64,
       grad_fn=<LinalgInvExBackward0>)


In [ ]:
b.sum().backward()
th_grad = a_tensor.grad
print(a_tensor.grad)
print(b.grad)

tensor([[-1.8489e-32, -1.1102e-16,  1.1102e-16],
        [-2.0817e-17, -2.5000e-01,  2.5000e-01],
        [ 2.0817e-17,  2.5000e-01, -2.5000e-01]], dtype=torch.float64)
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [ ]:
print((-b @ b.grad @ b).T)

tensor([[-1.8489e-32, -1.1102e-16,  1.1102e-16],
        [-2.0817e-17, -2.5000e-01,  2.5000e-01],
        [ 2.0817e-17,  2.5000e-01, -2.5000e-01]], dtype=torch.float64,
       grad_fn=<PermuteBackward0>)


### Needle example

In [ ]:
nd_a = ndl.Tensor(x)

inv_a = ndl.inv(nd_a)

In [ ]:
print(inv_a)

[[ 0.5000001  -0.625       0.12499997]
 [-1.         -0.75        0.74999994]
 [ 0.49999994  1.125      -0.62499994]]


In [ ]:
inv_a.sum().backward()
print(inv_a.grad)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [ ]:
print("nd:\n", nd_a.grad)
print("th:\n", th_grad)

nd:
 [[-5.3290705e-15  5.9604645e-08 -5.9604645e-08]
 [ 2.2351742e-08 -2.5000000e-01  2.5000000e-01]
 [-2.2351740e-08  2.4999994e-01 -2.4999994e-01]]
th:
 tensor([[-1.8489e-32, -1.1102e-16,  1.1102e-16],
        [-2.0817e-17, -2.5000e-01,  2.5000e-01],
        [ 2.0817e-17,  2.5000e-01, -2.5000e-01]], dtype=torch.float64)


In [ ]:
a = np.random.rand(10, 10) + np.eye(10)

In [ ]:
torch.set_printoptions(precision=8)

In [ ]:
th_tensor = torch.tensor(a, requires_grad=True)
th_inv = torch.inverse(th_tensor)
th_inv.sum().backward()
print(th_tensor.grad)

tensor([[-0.03958470, -0.07398275, -0.09110454, -0.02081114, -0.06773881,
         -0.04064466, -0.02585923, -0.03307357, -0.07176186, -0.03819216],
        [-0.03970795, -0.07421309, -0.09138819, -0.02087594, -0.06794971,
         -0.04077120, -0.02593974, -0.03317655, -0.07198529, -0.03831107],
        [-0.01288292, -0.02407784, -0.02965016, -0.00677303, -0.02204574,
         -0.01322789, -0.00841594, -0.01076386, -0.02335505, -0.01242972],
        [ 0.01443862,  0.02698540,  0.03323061,  0.00759092,  0.02470790,
          0.01482524,  0.00943222,  0.01206367,  0.02617532,  0.01393069],
        [-0.02144587, -0.04008175, -0.04935785, -0.01127489, -0.03669896,
         -0.02202012, -0.01400980, -0.01791832, -0.03887854, -0.02069143],
        [-0.03384819, -0.06326136, -0.07790190, -0.01779524, -0.05792227,
         -0.03475453, -0.02211177, -0.02828063, -0.06136231, -0.03265745],
        [-0.03160801, -0.05907453, -0.07274612, -0.01661750, -0.05408880,
         -0.03245438, -0.0206483

In [ ]:
nd_tensor = ndl.Tensor(a)
nd_inv = ndl.inv(nd_tensor)
nd_inv.sum().backward()
print(nd_tensor.grad)

[[-0.03958471 -0.07398275 -0.09110451 -0.02081114 -0.06773877 -0.04064467
  -0.02585924 -0.03307357 -0.07176187 -0.03819219]
 [-0.03970795 -0.0742131  -0.09138817 -0.02087597 -0.06794967 -0.04077118
  -0.02593973 -0.03317651 -0.07198524 -0.03831108]
 [-0.01288293 -0.02407785 -0.02965017 -0.00677304 -0.02204572 -0.01322789
  -0.00841594 -0.01076386 -0.02335505 -0.01242973]
 [ 0.01443863  0.0269854   0.03323061  0.00759095  0.02470788  0.01482523
   0.00943221  0.01206365  0.02617531  0.01393069]
 [-0.02144587 -0.04008176 -0.04935788 -0.01127496 -0.03669891 -0.02202009
  -0.01400978 -0.01791829 -0.03887854 -0.02069142]
 [-0.03384819 -0.06326136 -0.0779019  -0.01779526 -0.05792224 -0.03475455
  -0.02211178 -0.02828062 -0.06136232 -0.03265747]
 [-0.03160802 -0.05907454 -0.07274611 -0.01661753 -0.05408877 -0.03245437
  -0.02064835 -0.02640892 -0.05730117 -0.0304961 ]
 [-0.03216061 -0.06010732 -0.0740179  -0.01690802 -0.05503439 -0.03302179
  -0.02100935 -0.02687064 -0.05830296 -0.03102926]


## Tries

In [ ]:
def concatenate(its: tuple, axis):
    # compute out shape
    it_shape = its[0].shape
    out_shape = list(it_shape)
    out_shape[axis] = 0

    for tensor in its:
        # check shape
        for i in range(len(tensor.shape)):
            if i == axis:
                out_shape[axis] += tensor.shape[i]
            else:
                assert tensor.shape[i] == it_shape[i], \
                f"shape on axis {i} must be eq, except shape on axis {axis}!"

    # create a empty array and set value
    ret_arr = empty(out_shape)
    raw_idxs = []
    for i in range(len(out_shape) - 1):
        # build indexes
        raw_idxs.append(slice(0, out_shape[i], 1))

    start_idx = 0
    for tensor in its:
        idxs = raw_idxs.copy()
        idxs.insert(axis, slice(start_idx, start_idx + tensor.shape[axis], 1))
        ret_arr[tuple(idxs)] = tensor
        start_idx += tensor.shape[axis]
    
    # return
    return ret_arr


def _upper_trangle(a):
    x = array(a.numpy())
    for i in range(x.shape[1]):
        scale = x[i, i]
        for j in range(i+1, x.shape[0]):
            x[j, :] = x[j, :] - (x[j, i] / scale).broadcast_to(x[i, :].shape) * x[i, :]

    return x


def det(x):
    x = _upper_trangle(x)
    prod = 1
    for i in range(x.shape[0]):
        prod *= x[i, i]
    return prod


def _adj(x):
    adj_x = full(x.shape, 0.0)

    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if i == 0 and j == 0:
                sub_block = [i+1:, j+1:]

            sub_block = x[:i, :j]
            if j + 1 < adj_x.shape[1]:
                sub_block = np.concatenate([sub_block, x[:i, j + 1:]], axis=1)
                if i + 1 < adj_x.shape[0]:
                    sub_block_temp = np.concatenate([x[i + 1:, :j], x[i + 1:, j + 1:]], axis=1)
                    sub_block = np.concatenate([sub_block, sub_block_temp], axis=0)
            else:
                if i + 1 < len(adj_x):
                    sub_block = np.concatenate([sub_block, x[i + 1:, :j]], axis=0)

            adj_x[j, i] = (-1) ** (i + 1 + j + 1) * det(sub_block)

    return adj_x


def inv(x):
    adj_x = _adj(x)
    return adj_x / det(x).broadcast_to(adj_x.shape)

In [ ]:
np.linalg.inv(x)

**Do not run the following code!**

## eig tries

In [ ]:
# example array
x = np.array([
    [3., 2., 3.],
    [2., 3., 4.],
    [6., 7., 8.]
])

In [ ]:
out = np.linalg.eig(x)
print(out[0])
print(out[1])

In [ ]:
class linalg:
    @staticmethod
    def eig(x):
        aug = np.concatenate([np.eye(x.shape[0]), x], axis=1)
        row, col = x.shape

        for i in range(row):
            aug[i, :] = aug[i, :] / aug[i, i]
            for j in range(i + 1, row):
                aug[j, :] = aug[j, :] - aug[j, i] / aug[i, i] * aug[i, :]

        for i in range(row - 1, 0, -1):
            for j in range(i - 1, -1, -1):
                aug[j, :] -= aug[j, i] * aug[i, :]

        return aug[:, col:]


In [ ]:
import torch

In [ ]:
# example array
x = np.array([
    [3., 2., 3.],
    [2., 3., 4.],
    [6., 7., 8.]
])

In [ ]:
th_tensor = torch.tensor(x, requires_grad=True)
out = torch.linalg.eig(th_tensor)

In [ ]:
print(th_tensor.grad)

In [ ]:
out

In [ ]:
out[0].sum().backward()

In [ ]:
th_tensor.grad

****

In [ ]:
import numpy as np

In [ ]:
a = np.array([
    [2, 2, 3],
    [0, 5, 6],
    [0, 0, 0]
])

In [ ]:
def trui_solve(a, v_idx):
    v = np.array((a.shape[0], 1))
    row, col = a.shape
    assert row == col
    n = col

    for i in range(n-1, -1, -1):
        if i == v_idx:
            v[i] = 1
        else:
            a[i, :] /= a[i, i]
        
        for j in range(i-1, -1, -1):
            a[j, :] -= a[j, i] * a[i, :]

    # collect value
    for i in range(0, n):
        if i != v_idx:
            v[i] = -a[n-1, i]
    return v